In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
csv_column_name = ['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
Species = ['Setosa', 'Versicolor','Virginica']

In [3]:
train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

In [4]:
train = pd.read_csv(train_path, names=csv_column_name, header=0)
test = pd.read_csv(test_path, names=csv_column_name, header=0)

In [5]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [6]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

In [7]:
train.shape

(120, 4)

In [8]:
def input_fn(features, labels, training = True, batch_size=256):
    # convert inputs to a dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
    
    # shuffle and report if it is in training mode
    if training:
        dataset = dataset.shuffle(1000).repeat()
        
    return dataset.batch(batch_size)

In [9]:
feature_columns = []
for key in train.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))
print(feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [10]:
# Building Model

classifier = tf.estimator.DNNClassifier(feature_columns= feature_columns, hidden_units=[30,15,10], n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\mdmuz\\AppData\\Local\\Temp\\tmp_vfcb0_6', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
classifier.train(input_fn = lambda: input_fn(train,train_y, training = True), steps = 5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\mdmuz\AppData\Local\Temp\tmp_vfcb0_6\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.1034291, step = 0
INFO:tensorflow:global_step/sec: 393.7
INFO:tensorflow:loss = 1.0249889, step = 100 (0.254 sec)
INFO:tensorflow:global_step/sec: 641.026
INFO:tensorflow:loss = 1.0005023, step = 200 (0.157 sec)
INFO

In [12]:
eval_result = classifier.evaluate(input_fn = lambda: input_fn(test,test_y, training = False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-02-12T13:49:18
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\mdmuz\AppData\Local\Temp\tmp_vfcb0_6\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.27899s
INFO:tensorflow:Finished evaluation at 2022-02-12-13:49:18
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.95, average_loss = 0.3542334, global_step = 5000, loss = 0.3542334
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\mdmuz\AppData\Local\Temp\tmp_vfcb0_6\model.ckpt-5000

Test set accuracy: 0.950



In [19]:
def input_Fn(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength','SepalWidth','PetalLength','PetalWidth']
predict = {} # create an empty dictionary

print('Please type numeric values as prompted')
for feature in features:
    valid = True
    while valid:
        val = input(feature + ": ")
        if not val.isdigit(): valid = False
        
    predict[feature] = [float(val)]
    
predictions = classifier.predict(input_fn = lambda: input_Fn(predict))

for pred_dict in predictions:
    print(pred_dict)
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    
    print('Prediction is "{}" ({:.1f}%)'.format(Species[class_id], 100*probability))

Please type numeric values as prompted
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\mdmuz\AppData\Local\Temp\tmp_vfcb0_6\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([2.4939187, 0.9009034, 0.9152842], dtype=float32), 'probabilities': array([0.70943713, 0.14423685, 0.14632607], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1, 2]), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Setosa" (70.9%)
